In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -U librecommender

In [3]:
import pandas as pd

# Load your dataset
data = pd.read_csv("/content/drive/MyDrive/cs247project/data/undersampled_df_threshold_7.csv")
data.shape

(4246524, 23)

In [4]:
from libreco.data import random_split, DatasetPure
from libreco.evaluation import evaluate

# LibRecommender module requires "user", "item", and "label" columns in input
data.rename(columns={"user_id_hash": "user", "target_id_hash": "item", "is_click": "label"}, inplace=True)

# LibRecommender module requires "user" and "item" to be first two columns of input
columns = ["user", "item"]
new_col_order = columns + [col for col in data.columns if col not in columns]
data = data[new_col_order]

# Split data into training, evaluation, and testing sets
train_data, eval_data, test_data = random_split(data, multi_ratios=[0.8, 0.1, 0.1])

# Build training set
train_data, data_info = DatasetPure.build_trainset(train_data)
eval_data = DatasetPure.build_evalset(eval_data)

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
def train_and_evaluate(model, train_data, eval_data, neg_sampling=False, threshold=0.5):
    # Train the model on the training data
    model.fit(train_data, neg_sampling=neg_sampling)

    # Evaluate the model on the evaluation data
    print(evaluate(model, eval_data, neg_sampling=neg_sampling, metrics=["loss"]))

    users = test_data["user"]
    items = test_data["item"]
    predictions = []
    for user, item in zip(users, items):
        prediction = model.predict(user, item)
        predictions.append(prediction)

    binary_preds = [1 if prediction >= threshold else 0 for prediction in predictions]
    target = test_data["label"]
    correct_predictions = 0
    correct_predictions += (binary_preds == target).sum().item()
    print(correct_predictions/len(target))

# SVD

In [6]:
from libreco.algorithms import SVD

# Create the SVD model instance
svd = SVD(task="rating", data_info=data_info, n_epochs=1)

# Train and evaluate the model
train_and_evaluate(svd, train_data, eval_data)

Training start time: 2024-03-17 01:37:41


train: 100%|██████████| 26541/26541 [02:27<00:00, 179.63it/s]


Epoch 1 elapsed: 147.773s


eval_pointwise: 100%|██████████| 52/52 [00:00<00:00, 368.47it/s]


{'loss': 0.43158945}
0.7196055736626216


# SVD++

In [6]:
from libreco.algorithms import SVDpp

# Create the SVD++ model instance
svdpp = SVDpp(task="rating", data_info=data_info, n_epochs=1)

# Train and evaluate the model
train_and_evaluate(svdpp, train_data, eval_data)

Training start time: 2024-03-17 01:42:05


train: 100%|██████████| 26541/26541 [07:22<00:00, 60.04it/s]


Epoch 1 elapsed: 442.087s


eval_pointwise: 100%|██████████| 52/52 [00:00<00:00, 222.77it/s]


{'loss': 0.43142688}
0.7194138517871556


# Wide & Deep

In [ ]:
from libreco.algorithms import WideDeep

# Create the WideDeep model instance
wd = WideDeep(task="rating", data_info=data_info, n_epochs=1)

# Train and evaluate the model
train_and_evaluate(wd, train_data, eval_data)

# DeepFM

In [ ]:
from libreco.algorithms import DeepFM

# Create the DeepFM model instance
dfm = DeepFM(task="rating", data_info=data_info, n_epochs=1)

# Train and evaluate the model
train_and_evaluate(dfm, train_data, eval_data)

# NCF

In [ ]:
from libreco.algorithms import NCF

# Create the NCF model instance
ncf = NCF(task="rating", data_info=data_info, n_epochs=1)

# Train and evaluate the model
train_and_evaluate(ncf, train_data, eval_data)

# Transformer

In [ ]:
from libreco.algorithms import Transformer

# Create the NCF model instance
tf = Transformer(task="rating", data_info=data_info, n_epochs=1)

# Train and evaluate the model
train_and_evaluate(tf, train_data, eval_data)